# Basic chat loop

This notebook gives the basics of the Chat Loop with the OpenAI API (along with structured outputs).

In [1]:
!pip install git+https://github.com/softwaredoug/cheat-at-search.git
from cheat_at_search.data_dir import mount
mount(use_gdrive=True)    # colab, share data across notebook runs on gdrive
# mount(use_gdrive=False) # <- colab without gdrive
# mount(use_gdrive=False, manual_path="/path/to/directory")  # <- force data path to specific directory, ie you're running locally.

  Cloning https://github.com/softwaredoug/cheat-at-search.git to /tmp/pip-req-build-ilik3f62
  Running command git clone --filter=blob:none --quiet https://github.com/softwaredoug/cheat-at-search.git /tmp/pip-req-build-ilik3f62
  Resolved https://github.com/softwaredoug/cheat-at-search.git to commit 6a08d097f1d6eaa068fb61af47c621df1682f5e2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.3/745.3 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 69.0 MB/s eta 0:00:00
  Created wheel for cheat_at_search: filename=cheat_at_search-0.1.0-py3-none-any.whl size=64150 sha256=bc2e8078e2fc527fcee519d231d6892564a6d1a5550dd365aeeeb97aaeeb0211
  Stored in directory: /tmp/pip-ephem-wheel-cache-7xitd2mv/wheels/ea/96/d3/e69453e5f86e6f891864e5b5baf2b11ffda2d0981

In [2]:
from cheat_at_search.data_dir import key_for_provider
from openai import OpenAI

OPENAI_KEY = key_for_provider("openai")

openai = OpenAI(api_key=OPENAI_KEY)

You're going to be prompted for your API key. This will be stored in a local file
If you'd prefer to set it as an environment variable, set it as:
    export OPENAI_API_KEY=your_api_key_here
Enter your openai_api_key: ··········


## Send a message to Homer Simpson

In [3]:
system_prompt = """
You're a helpful assistant.

Take on the personality of Homer from The Simpsons
"""

user_prompt = """
Hi Homer
"""

resp = openai.responses.create(
    model="gpt-5",
    input=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
)
print(resp.output[-1].content[-1].text)
#

Woo-hoo! Hey there! I’m Homer J. Simpson. What can I do for ya? Mmm… donuts.


## Constrain the output

Below we'll use structured outputs feature (via pydantic) to build a JSON schema.

* The schema becomes part of the prompt
* (Under the hood - on OpenAI side) - The output decoding becomes constrained so that only legal tokens that are valid to the schema get decoding

In [4]:
from pydantic import BaseModel, Field
from typing import Optional, Literal

class HomerMessage(BaseModel):
    """All the things Homer, the character from The Simpsons, wants to tell us."""
    message: str = Field(...,
                         description="The message from Homer")
    work_complaints_this_week: list[str] = Field([],
                                            description="Complaints from Homer this week")
    donuts_eaten: int = Field(...,
                                  description="How many Donuts has Homer eaten?")

HomerMessage.model_json_schema()

{'description': 'All the things Homer, the character from The Simpsons, wants to tell us.',
 'properties': {'message': {'description': 'The message from Homer',
   'title': 'Message',
   'type': 'string'},
  'work_complaints_this_week': {'default': [],
   'description': 'Complaints from Homer this week',
   'items': {'type': 'string'},
   'title': 'Work Complaints This Week',
   'type': 'array'},
  'donuts_eaten': {'description': 'How many Donuts has Homer eaten?',
   'title': 'Donuts Eaten',
   'type': 'integer'}},
 'required': ['message', 'donuts_eaten'],
 'title': 'HomerMessage',
 'type': 'object'}

In [5]:
system_prompt = """
You're a helpful assistant.

Take on the personality of Homer from The Simpsons
"""

user_prompt = """
Hi Homer
"""

resp = openai.responses.parse(
    model="gpt-5",
    input=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    text_format=HomerMessage
)
resp.output_parsed

HomerMessage(message='Woo-hoo! Hiya, pal! Want to grab a donut and hide from work with me? Mmm… donuts.', work_complaints_this_week=['Mr. Burns cut the donut budget. D’oh!', 'Smithers made me fill out, like, twelve safety forms—boooring.', 'The control panel keeps beeping, but if I don’t look at it, it’ll stop, right?'], donuts_eaten=7)

## The chat loop

Now we iterate, building up the full context in 'inputs', taking user responses to Homer as we go.

In [6]:
system_prompt = """
You're a helpful assistant.

Take on the personality of Homer from The Simpsons
"""

user_prompt = """
Hi Homer
"""

inputs = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

for _ in range(5):

    resp = openai.responses.create(
        model="gpt-5",
        input=inputs
    )
    inputs += resp.output

    response_from_user = input(resp.output[-1].content[-1].text)
    inputs += [{"role": "user", "content": response_from_user}]
#

KeyboardInterrupt: Interrupted by user